In [1]:
# import modules
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from math import sqrt

In [40]:
class Pattern:
    def __init__(self, pattern_size, square_size, type):
        self.pattern_size = pattern_size
        self.square_size = square_size
        self.type = type
        
        return
    
    def construct3DPoints(self):
        board_points = np.zeros((self.pattern_size[0] * self.pattern_size[1], 3), np.float32)
        board_points[:,:2] = np.mgrid[ 0:self.pattern_size[0], 0:self.pattern_size[1] ].T.reshape(-1,2)
        board_points = board_points * self.square_size
        
        return board_points

In [39]:
class CornerDetector:
    def __init__(self, image_path, pattern_size):
        self.image_path = image_path
        self.pattern_size = pattern_size
        self.criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        self.image_points = []
        self.image_size = 0
        return
    
    def detect_using_SB(self):
        # Create empty variables
        images_detected = 0
        
        # For each image in directory
        for image_file in self.image_path:
            # Read the image file
            img = cv2.imread(image_file)
            # Convert to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Return the image size
            self.image_size = gray.shape[::-1]
            # Detect the corners in images
            detected, corners = cv2.findChessboardCornersSB(gray, self.pattern_size, None)
            
            if detected:
                # Save corners in image points array
                self.image_points.append(corners)
                images_detected += 1
                
        return images_detected
    def detect_using_subpixel(self):
        # Create empty variables
        images_detected = 0
        
        # For each image in directory
        for image_file in self.image_path:
            # Read the image file
            img = cv2.imread(image_file)
            # Convert to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Return the image size
            self.image_size = gray.shape[::-1]
            # Detect the corners in images
            detected, corners = cv2.findChessboardCorners(gray, self.pattern_size, None)
            
            if detected:
                corners = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), self.criteria)
                # Save corners in image points array
                self.image_points.append(corners)
                images_detected += 1
                
            return images_detected            

In [ ]:
# Choose the image path
path = '../../../images/virtual/'
images = glob(path + '7x7/opengl3/*.jpg')

myCornerDetector = CornerDetector(images, (7,7))

myCornerDetector.detect_using_SB()

In [2]:
class Calibrator:
    def __init__(self, images_path, image_size, pattern_size, square_size, flags_calib):
        # Calib config
        self.pattern_size = pattern_size
        self.square_size = square_size
        self.images_path = images_path
        self.image_size = image_size
        self.flags_calib = flags_calib

        # Data points
        self.world_points = []
        self.image_points = []

        # Calib results
        self.results = {}
        
    # Generate board points
    def construct3DPoints(self):
        
        board_points = np.zeros((self.pattern_size[0] * self.pattern_size[1], 3), np.float32)
        board_points[:,:2] = np.mgrid[ 0:self.pattern_size[0], 0:self.pattern_size[1] ].T.reshape(-1,2)
        board_points = board_points * self.square_size
        
        return board_points
    
    def calibrate(self):
        
        # Generate board points
        board_points = np.zeros((self.pattern_size[0] * self.pattern_size[1], 3), np.float32)
        board_points[:,:2] = np.mgrid[ 0:self.pattern_size[0], 0:self.pattern_size[1] ].T.reshape(-1,2)
        board_points = board_points * self.square_size
        
        # Detect corners in chessboard
        counter = 0
        for fname in self.images_path:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            self.img_size = gray.shape[::-1]
            detected, corners = cv2.findChessboardCornersSB(gray, self.pattern_size, None)
            if detected:
                self.world_points.append(board_points)
                self.image_points.append(corners)
                counter+=1

        print("Corners found in " + str(counter) + " images")

        # Calibrate the camera
        rms, camera_matrix, distortion_coeffs, rvecs, tvecs, std_intrinsic, std_extrinsic, per_view_error = cv2.calibrateCameraExtended(self.world_points, self.image_points, self.image_size, None, None,
                                                                flags=self.flags_calib)

        print("RMS re-projection error:", rms)
        print("The median re-projection error", np.median(per_view_error))
        print("Camera Matrix:\n", camera_matrix)
        print("Distortion Parameters:\n", distortion_coeffs)
        
        # Save the calibration results
        self.results = {
        "error_rms": rms,
        "camera_matrix": camera_matrix,
        "distortion_coeffs": distortion_coeffs,
        "rvecs": rvecs,
        "tvecs": tvecs,
        "std_intrinsic": std_intrinsic,
        "std_extrinsic": std_extrinsic,
        "per_view_error": per_view_error
        }

        return 

In [ ]:
def calculate_mean_error(image_set, pattern_size, square_size, camera_matrix, distortion, per_view_error = False):
    
    # Create empty arrays for rotation and translation vectors
    rvecs = []
    tvecs = []
    errors = []
    
    # Define the world board points
    board_points = construct3DPoints(pattern_size, square_size)
    board_points = board_points.astype('float32')
    
    # Calculate image points in each image
    for image in image_set:
        img = cv2.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect chessboard corners in the image
        ret, detected_image_points = cv2.findChessboardCornersSB(gray, pattern_size, None)
        
        # If it was detected
        if ret == True: 

            # Calculate extrinsic parameters 
            _, rvec, tvec = cv2.solvePnP(board_points, detected_image_points, camera_matrix, distortion)
            rvecs.append(rvec)
            tvecs.append(tvecs)

            # Calculate projected image points
            projected_image_points, _ = cv2.projectPoints(board_points, rvec, tvec, camera_matrix, distortion)

            # Find the Euclidean Distance between projected and detected image points
            error = cv2.norm(detected_image_points, projected_image_points, normType= cv2.NORM_L2) / len(projected_image_points)
            
            errors.append(error)
            
            if per_view_error:
                print(f"Mean Error of Image {image}:", error, "px")
    
    mean_error = np.median(errors)
    
    print("Median error: ", mean_error)
    
    return mean_error

In [3]:


counter = 0
image_set = []

i = np.random.choice(np.arange(0,49),10, replace=False)

for idx in i:

    image_set.append(images[idx])

# Choose the calibration flags
flagsCalib = cv2.CALIB_RATIONAL_MODEL

In [4]:
myCameraCalbrator = Calibrator(images_path = image_set, image_size = (720,720), pattern_size = (7,7), square_size = 30, flags_calib = flagsCalib)

myCameraCalbrator.calibrate()

Corners found in 10 images
RMS re-projection error: 0.25237644703787737
The median re-projection error 0.2312917182726287
Camera Matrix:
 [[618.96517954   0.         360.15005095]
 [  0.         618.84831606 358.79777245]
 [  0.           0.           1.        ]]
Distortion Parameters:
 [[-3.45848564e+00 -1.00598529e+00 -2.39464350e-04  6.23250076e-04
   8.47311744e+00 -3.45808192e+00 -1.01409630e+00  8.49145474e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [8]:
myCameraCalbrator.results

{'error_rms': 0.25237644703787737,
 'camera_matrix': array([[618.96517954,   0.        , 360.15005095],
        [  0.        , 618.84831606, 358.79777245],
        [  0.        ,   0.        ,   1.        ]]),
 'distortion_coeffs': array([[-3.45848564e+00, -1.00598529e+00, -2.39464350e-04,
          6.23250076e-04,  8.47311744e+00, -3.45808192e+00,
         -1.01409630e+00,  8.49145474e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00]]),
 'rvecs': (array([[-0.17003392],
         [-0.17574791],
         [-0.01527335]]),
  array([[ 0.00122555],
         [-0.00443264],
         [-0.00037959]]),
  array([[ 0.17457668],
         [-0.17513512],
         [ 0.01520712]]),
  array([[0.00351786],
         [0.1762347 ],
         [0.00040202]]),
  array([[ 0.17490071],
         [-0.17525834],
         [ 0.01551227]]),
  array([[ 0.17554436],
         [-0.17389668],
         [ 0.01539129]]),
  array([[ 0.17409377],
         